# Setup

In [ ]:
!pip install scipy==1.6.3
!pip install scikit_optimize==0.8.1
!pip install scikit_learn==0.24.2
! git clone https://github.com/zykhoo/predicting_hamiltonian_dynamics.git

     |████████████████████████████████| 27.4 MB 1.5 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 101 kB 4.6 MB/s 
     |████████████████████████████████| 22.3 MB 1.4 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but y

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Run all

In [ ]:
"""# Experiment setup"""

import numpy as np

experiment,sys,dim = "NN","inversecos",2

H = lambda x: np.arccos(x[1]*x[0]**3)
f1 = lambda x: -x[0]**3 / np.sqrt(1-x[0]**6 * x[1]**2) # dH/dy
f2 = lambda x: 3*x[1]*x[0]**2 / np.sqrt(1-x[1]**2 * x[0]**6) # -dH/dx
spacedim = [(-0.9,0.9),(-0.9,0.9)]
x0,H0=1.,np.pi/2

h= 0.1
ini = initialcon = [64, 128, 256, 512, 1024]
LR = 0.01
diagdist = np.sum(np.square(np.asarray([spacedim[0][0], spacedim[0][1]]), np.asarray([spacedim[1][0], spacedim[1][1]])))
epsilon = 0.1
fvector = lambda x: np.asarray([f1(x), f2(x)])


"""# Test dataset creation"""

from predicting_hamiltonian_dynamics import groundtruth_2dim
from tqdm import tqdm
import time 

evaluation_length_long = 50
evaluation_length_one = 1

xxlong,yylong = np.linspace(spacedim[0][0], spacedim[0][1], 5), np.linspace(spacedim[1][0], spacedim[1][1], 5)
xlong,ylong = np.meshgrid(xxlong,yylong)
xxshort,yyshort = np.linspace(spacedim[0][0], spacedim[0][1], 20), np.linspace(spacedim[1][0], spacedim[1][1], 20)
xshort,yshort = np.meshgrid(xxshort,yyshort)

trajectories_groundtruth_start_long = np.expand_dims(groundtruth_2dim.classicTrajectory(np.asarray([[0.4],[0.]]),f1,f2,h = 0.1,N=evaluation_length_long,n_h = 1),0)
for i in tqdm(np.expand_dims(np.c_[np.ravel(xlong),np.ravel(ylong)],2)):
  results_200 = np.expand_dims(groundtruth_2dim.classicTrajectory(i,f1,f2,h = 0.1,N=evaluation_length_long,n_h = 1),0)
  trajectories_groundtruth_start_long = np.vstack((trajectories_groundtruth_start_long, results_200))
trajectories_groundtruth_start_long.shape # output no., start and final, p and q, full traj

trajectories_groundtruth_start_short = np.expand_dims(groundtruth_2dim.classicTrajectory(np.asarray([[0.4],[0.]]),f1,f2,h = 0.1,N=evaluation_length_one,n_h = 1),0)
for i in tqdm(np.expand_dims(np.c_[np.ravel(xshort),np.ravel(yshort)],2)):
  results_200 = np.expand_dims(groundtruth_2dim.classicTrajectory(i,f1,f2,h = 0.1,N=evaluation_length_one,n_h = 1),0)
  trajectories_groundtruth_start_short = np.vstack((trajectories_groundtruth_start_short, results_200))
trajectories_groundtruth_start_short.shape # output no., start and final, p and q, full traj

within_array = groundtruth_2dim.get_within_array(trajectories_groundtruth_start_long, spacedim)

100%|██████████| 400/400 [00:00<00:00, 1029.03it/s]


In [ ]:
path = '/content/drive/MyDrive/SSI/Baseline v2/synthetic systems (upload)/'

In [ ]:
from predicting_hamiltonian_dynamics.models import NN_2dim
from predicting_hamiltonian_dynamics.models import GP_2dim
from predicting_hamiltonian_dynamics.models import PINN_2dim
from predicting_hamiltonian_dynamics.models import PIGP_2dim


for i in range(20):
  seed = i
  np.random.seed(seed=seed)
  for ini in initialcon: 

    start, final = groundtruth_2dim.CreateTrainingDataTrajClassicIntRandom(1,ini,spacedim,h,f1,f2,seed = seed,n_h = 1)

    delta = start.copy()
    delta[0,:] = f1(start)
    delta[1,:] = f2(start)

    """# NN"""

    from predicting_hamiltonian_dynamics.models import NN_2dim
    import torch

    if torch.cuda.is_available():
      device=torch.device('cuda')
    else:
      device=torch.device('cpu')
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


    wholemat, evalmat = NN_2dim.data_preprocessing(start, delta, device)

    import torch.optim as optim
    import time 

    net = NN_2dim.Net(dim,16,dim)
    starttime = time.time() 
    net = NN_2dim.train(net, wholemat, evalmat, optimizer=optim.Adam(net.parameters(), lr=LR), batchsize=10, iter=1600, )
    traintime = time.time()-starttime


    from tqdm import tqdm
    from predicting_hamiltonian_dynamics.models import NN_2dim
    from predicting_hamiltonian_dynamics import metrics


    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = NN_2dim.compute_metrics_NN(net, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)

    file_object = open(path + 'InverseCos_random.txt', 'a')
    file_object.write('NN, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()


    """# GP"""

    from sklearn.gaussian_process import GaussianProcessRegressor
    from sklearn.gaussian_process.kernels import RBF
    from sklearn import preprocessing

    output = delta
    scaler = preprocessing.MinMaxScaler()
    input = scaler.fit_transform(start.transpose())
    kernel = 1 * RBF(length_scale=2., length_scale_bounds="fixed")
    gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=0, optimizer = None, random_state = seed)
    starttime = time.time() 
    gaussian_process.fit(input, output.transpose())
    traintime = time.time()-starttime
    gaussian_process.predict(scaler.transform([[0.7,0.8]]))

    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = GP_2dim.compute_metrics_GP(gaussian_process, scaler, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)

    file_object = open(path + 'InverseCos_random.txt', 'a')
    file_object.write('GP, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()


    """# PINN"""

    from predicting_hamiltonian_dynamics.models import PINN_2dim
    import torch

    if torch.cuda.is_available():
      device=torch.device('cuda')
    else:
      device=torch.device('cpu')
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    wholemat, evalmat = PINN_2dim.data_preprocessing(start, delta,device)
    net=PINN_2dim.Net(dim,16,1)
    starttime = time.time()
    net,avg_vallosses,avg_lossli,avg_f1li,avg_f2li,avg_f3li,avg_f4li=PINN_2dim.train(net,bs=10,num_epoch=5000,initial_conditions=initialcon,device=device,
                                                                                    wholemat=wholemat,evalmat=evalmat,x0=x0,H0=H0,dim=dim,LR=LR,patience=500,c1=1,c2=1,c3=1,c4=1)
    traintime = time.time()-starttime

    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = PINN_2dim.compute_metrics_PINN(net, device, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)
    
    file_object = open(path + 'InverseCos_random.txt', 'a')
    file_object.write('PINN, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()

    """# PIGP"""

    from predicting_hamiltonian_dynamics.models import PIGP_2dim
    GP = PIGP_2dim.BertalanGP()
    starttime = time.time()
    GP.train(start,delta,h)
    traintime = time.time()-starttime

    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = PIGP_2dim.compute_metrics_PIGP(GP, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)

    file_object = open(path + 'InverseCos_random.txt', 'a')
    file_object.write('PIGP, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()


training loss tensor(0.2142, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.1358, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.0842, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.0302, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.0032, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.0033, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.0007, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.0012, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.0004, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.0008, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.0003, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.0005, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.0003, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation lo

  0%|          | 0/50 [00:00<?, ?it/s]/content/predicting_hamiltonian_dynamics/models/NN_2dim.py:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  fstage = lambda stg: h * torch.squeeze(net(torch.tensor([q+stg,p]).float().transpose(1,0)),0).detach().numpy().transpose()[:dim]
100%|██████████| 50/50 [00:00<00:00, 93.69it/s]

100%|██████████| 50/50 [00:00<00:00, 2866.29it/s]

100%|██████████| 50/50 [00:00<00:00, 233.90it/s]

100%|██████████| 50/50 [00:00<00:00, 225.36it/s]

100%|██████████| 50/50 [00:00<00:00, 369.81it/s]

100%|██████████| 50/50 [00:00<00:00, 316.65it/s]

100%|██████████| 50/50 [00:00<00:00, 3555.04it/s]

100%|██████████| 50/50 [00:00<00:00, 327.75it/s]

100%|██████████| 50/50 [00:00<00:00, 599.28it/s]

100%|██████████| 50/50 [00:00<00:00, 270.65it/s]

100%|███

 
epoch= 0  time= 0.023431062698364258  loss= 1.9074065685272217  val_loss= 0.8112172186374664  f1= 0.08435227631385027  f2= 0.34604596294366474  f3= 1.4745795081059139  f4= 0.0024288143897004144 percent lr= 0.01
 
epoch= 10  time= 0.17743825912475586  loss= 0.32892146388379234  val_loss= 0.36136260628700256  f1= 0.054143222959465054  f2= 0.2712009460882342  f3= 0.0003839878493939371  f4= 0.0031933086364447287 percent lr= 0.01
 
epoch= 20  time= 0.31406331062316895  loss= 0.5122880563139915  val_loss= 0.29961390048265457  f1= 0.05174979934513566  f2= 0.44001054404406476  f3= 0.0002125633615529902  f4= 0.020315161123457503 percent lr= 0.01
 
epoch= 30  time= 0.4605991840362549  loss= 0.28451191013058025  val_loss= 0.20452741533517838  f1= 0.013093407392440111  f2= 0.23596380743080606  f3= 0.00030681697677673964  f4= 0.0351478802055738 percent lr= 0.01
 
epoch= 40  time= 0.6031134128570557  loss= 0.20522081603606543  val_loss= 0.16610384359955788  f1= 0.014612306719859048  f2= 0.16968279

100%|██████████| 400/400 [00:02<00:00, 196.10it/s]


Start training with for 64 data points
Start computation of covariance matrix.
Covariance matrix of shape (64, 64)computed.
Start Cholesky decomposition of (64, 64) Matrix
Cholesky decomposition completed.
Create LHS of linear system for H at test points.
Creation of linear system completed.
Solve least square problem of dimension (129, 64)


...

100%|██████████| 50/50 [00:00<00:00, 128.61it/s]

100%|██████████| 50/50 [00:00<00:00, 1892.16it/s]

100%|██████████| 50/50 [00:00<00:00, 140.86it/s]

100%|██████████| 50/50 [00:00<00:00, 83.87it/s]

100%|██████████| 50/50 [00:00<00:00, 593.42it/s]

100%|██████████| 50/50 [00:00<00:00, 256.06it/s]

100%|██████████| 50/50 [00:00<00:00, 2622.23it/s]

100%|██████████| 50/50 [00:00<00:00, 262.26it/s]

100%|██████████| 50/50 [00:00<00:00, 127.80it/s]

100%|██████████| 50/50 [00:00<00:00, 86.26it/s]

100%|██████████| 50/50 [00:00<00:00, 235.03it/s]

100%|██████████| 50/50 [00:00<00:00, 2883.63it/s]

100%|██████████| 4/4 [00:00<00:00, 418.20it/s]

100%|██████████| 50/50 [00:00<00:00, 301.85it/s]

100%|██████████| 50/50 [00:00<00:00, 3011.29it/s]

100%|██████████| 15/15 [00:00<00:00, 325.32it/s]

100%|██████████| 50/50 [00:00<00:00, 206.91it/s]

100%|██████████| 50/50 [00:00<00:00, 3135.18it/s]

100%|██████████| 50/50 [00:00<00:00, 213.37it/s]

100%|██████████| 50/50 [00:00<00:00, 157.81it/s]


 
epoch= 0  time= 0.2212052345275879  loss= 0.4198729674354559  val_loss= 0.293219278100878  f1= 0.06478682538243546  f2= 0.31295522969601475  f3= 0.04075083256260028  f4= 0.0013800816670373876 percent lr= 0.01
 
epoch= 10  time= 2.613856554031372  loss= 0.11245046958054711  val_loss= 0.12483786600641907  f1= 0.01622449391026705  f2= 0.08878523676863877  f3= 0.0012058107313329977  f4= 0.00623492902155688 percent lr= 0.01
 
epoch= 20  time= 5.230693578720093  loss= 0.004933383084390676  val_loss= 0.005633127226610668  f1= 0.0010848269717666722  f2= 0.002690473612771842  f3= 0.0007108834306335189  f4= 0.00044719905785850636 percent lr= 0.01
 
epoch= 30  time= 7.779421329498291  loss= 0.002286222737450039  val_loss= 0.0021640766339260154  f1= 0.0004268637843546053  f2= 0.0012394042632433366  f3= 0.0004492355847972016  f4= 0.00017071911384146043 percent lr= 0.01
 
epoch= 40  time= 10.456473588943481  loss= 0.002199013120489104  val_loss= 0.0016090428063762375  f1= 0.00043051494884171386  f

100%|██████████| 400/400 [00:03<00:00, 131.90it/s]


Start training with for 1024 data points
Start computation of covariance matrix.
Covariance matrix of shape (1024, 1024)computed.
Start Cholesky decomposition of (1024, 1024) Matrix
Cholesky decomposition completed.
Create LHS of linear system for H at test points.
Creation of linear system completed.
Solve least square problem of dimension (2049, 1024)


100%|██████████| 400/400 [08:50<00:00,  1.33s/it]
